# add fiber assign weights

In [1]:
import os
import h5py
import fitsio
import numpy as np
from tqdm import tqdm
from scipy.special import erf
import astropy.table as aTable

In [2]:
import desitarget.io
from desitarget.sv3.sv3_targetmask import bgs_mask as sv3_bgs_mask


Bad key text.latex.preview in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95 

In [3]:
import matplotlib.pyplot as plt

## read in target catalog

In [4]:
ff = fitsio.read('/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/fuji/LSScats/EDAbeta/BGS_ANY_full.dat.fits')

# select only BGS targets
is_bgs_bright = (
    (ff['COADD_FIBERSTATUS'] == 0) & 
    ((ff['SV3_BGS_TARGET'] & sv3_bgs_mask['BGS_BRIGHT']) != 0) & 
    (ff['SPECTYPE'] == 'GALAXY') & 
    (ff['Z_HP'] > 0.) & (ff['Z_HP'] < 0.6) & 
    (ff['ZWARN'] == 0) & 
    (ff['DELTACHI2'] > 40.) & 
    (ff['ZERR'] < 0.0005 * (1 + ff['Z_HP'])))

is_bgs_faint = (
    (ff['COADD_FIBERSTATUS'] == 0) & 
    ((ff['SV3_BGS_TARGET'] & sv3_bgs_mask['BGS_FAINT']) != 0) & 
    (ff['SPECTYPE'] == 'GALAXY') & 
    (ff['Z_HP'] > 0.) & (ff['Z_HP'] < 0.6) & 
    (ff['ZWARN'] == 0) & 
    (ff['DELTACHI2'] > 40.) & 
    (ff['ZERR'] < 0.0005 * (1 + ff['Z_HP'])))

print('%i BGS BRIGHT z-success' % np.sum(is_bgs_bright))
print('%i BGS FAINT z-success' % np.sum(is_bgs_faint))

143074 BGS BRIGHT z-success
96771 BGS FAINT z-success


## read in VAGC

In [5]:
fvagc = '/global/cfs/cdirs/desi/users/chahah/provabgs/svda/BGS_ANY_full.provabgs.hdf5'
vagc = aTable.Table.read(fvagc)
vagc[:5]

has_va = (vagc['provabgs_z_max'] != -999.)

# assign fiber assign weights using MTLS

In [6]:
with open('/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/altmtl/SV3HPList.txt') as f: 
    hplist = [int(hp) for hp in f.readline().split(',')]

In [8]:
target_ids = ff['TARGETID']

nobs = np.zeros((len(target_ids), 128))
for imtl in tqdm(range(128)): 
    # compile mtls
    mtls = []
    for pix in hplist:
        targs = desitarget.io.read_mtl_ledger(
            os.path.join('/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/altmtl/DESI_EDA_SV3AltMTLs/', 
                         'Univ%s' % str(imtl).zfill(3), 'sv3', 'bright', 'sv3mtl-bright-hp-%i.ecsv' % pix), 
            unique=True, isodate=None, initial=False, leq=False)
        mtls.append(targs)
    mtls = np.concatenate(mtls)
    mtls = mtls[mtls['NUMOBS'] > 0.5]
    
    fa_ed = np.in1d(target_ids, mtls['TARGETID'])
    nobs[fa_ed,imtl] = 1.

[False  True False ...  True  True  True]
[False False False ...  True False  True]
[False False False ...  True  True  True]
[False False False ...  True  True  True]
[False  True False ...  True  True  True]
[False False False ...  True  True  True]
[False False False ...  True  True  True]
[False False False ...  True False  True]
[False False False ...  True  True  True]
[False False False ...  True  True  True]
[False False False ...  True False  True]
[False False False ...  True  True  True]
[False  True False ...  True  True  True]
[False False False ...  True  True  True]
[False  True False ...  True  True  True]
[False False False ...  True  True  True]
[False  True False ...  True  True  True]
[False  True False ...  True  True  True]
[False False False ...  True  True  True]
[False  True False ...  True  True  True]
[False  True False ...  True  True  True]
[False False False ...  True  True  True]
[False False False ...  True  True  True]
[False False False ...  True  True

In [9]:
nobs_mtls = np.sum(nobs, axis=1)

In [11]:
w_fa = 129. / (nobs_mtls + 1) # calculate the weights

In [15]:
print(w_fa[is_bgs_bright].min(), w_fa[is_bgs_bright].max())
print(w_fa[is_bgs_faint].min(), w_fa[is_bgs_faint].max())

1.0 10.75
1.0 129.0


In [17]:
vagc.add_column('provabgs_w_fibassign')
vagc['provabgs_w_fibassign'] = w_fa

In [18]:
vagc.write(fvagc, overwrite=True)

/global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/astropy/io/misc/hdf5.py:283: UserWarning: table path was not set via the path= argument; using default path __astropy_table__
  "using default path {}".format(path))
